In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 


import gc
import time
import numpy as np
import pandas as pd
from sklearn.cross_validation import train_test_split
import lightgbm as lgbm
#from xgboost import plot_importance
import matplotlib.pyplot as plt
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os

# Any results you write to the current directory are saved as output.

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/matplotlib/__init__.py:1067: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #2
  (fname, cnt))
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/matplotlib/__init__.py:1067: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #3
  (fname, cnt))
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
%%time
is_valid = False

path="./mnt/ssd/kaggle-talkingdata2/competition_files/"

train_columns = ['ip', 'app', 'device', 'os', 'channel', 'click_time', 'is_attributed']
#test_columns  = ['ip', 'app', 'device', 'os', 'channel', 'click_time']
dtypes = {
        'ip'            : 'uint32',
        'app'           : 'uint16',
        'device'        : 'uint16',
        'os'            : 'uint16',
        'channel'       : 'uint16',
        'is_attributed' : 'uint8',
        'click_id'      : 'uint32'
        }

# Read the last lines because they are more impacting in training than the starting lines
##train = pd.read_csv(path+"train.csv", skiprows=range(1,131886954), nrows=53016937, usecols=train_columns, dtype=dtypes)
train = pd.read_csv(path+"train.csv", skiprows=range(1,74903891), nrows=110000000, usecols=train_columns, dtype=dtypes)

#valid = pd.read_csv(path+"test.csv", usecols=test_columns, dtype=dtypes)


CPU times: user 1min 50s, sys: 3.53 s, total: 1min 54s
Wall time: 1min 54s


In [3]:

gc.collect()

7

In [ ]:
#Y=train.is_attributed
#train.drop('is_attributed',axis=1,inplace=True)

In [4]:
def timeFeatures(df,colname):
    # Make some new features with click_time column
    df['datetime'] = pd.to_datetime(df[colname])
    df['dow']      = df['datetime'].dt.dayofweek
    df["hour"]      = df['datetime'].dt.hour
    #df["dteom"]    = df["datetime"].dt.daysinmonth - df["datetime"].dt.day
    df.drop(['datetime'], axis=1, inplace=True)
    return df


In [5]:
%%time
#valid=timeFeatures(valid)
train=timeFeatures(train,'click_time')
gc.collect()

CPU times: user 41.8 s, sys: 3.64 s, total: 45.4 s
Wall time: 44.8 s


In [6]:
train.head()

,ip,app,device,os,channel,click_time,is_attributed,dow,hour
0,87242,24,1,40,178,2017-11-08 01:47:40,0,2,1
1,49565,12,1,9,259,2017-11-08 01:47:40,0,2,1
2,39785,9,1,13,244,2017-11-08 01:47:40,0,2,1
3,5348,3,1,18,280,2017-11-08 01:47:40,0,2,1
4,56516,9,1,17,466,2017-11-08 01:47:40,0,2,1


In [ ]:
'''
nrow_train = train.shape[0]
merge = pd.concat([train, test])
gc.collect()
'''

In [7]:
valid=pd.DataFrame()
valid=train[train.dow==3]
train=train[train.dow!=3]
gc.collect()
valid.shape,train.shape

((53016937, 9), (56983063, 9))

In [8]:
train.head()

,ip,app,device,os,channel,click_time,is_attributed,dow,hour
0,87242,24,1,40,178,2017-11-08 01:47:40,0,2,1
1,49565,12,1,9,259,2017-11-08 01:47:40,0,2,1
2,39785,9,1,13,244,2017-11-08 01:47:40,0,2,1
3,5348,3,1,18,280,2017-11-08 01:47:40,0,2,1
4,56516,9,1,17,466,2017-11-08 01:47:40,0,2,1


In [11]:
%%time
###Experiment
def get_feature_count(train_df,valid_df):
    

    nrow_train = train_df.shape[0]
    df = pd.concat([train_df, valid_df])
    gc.collect()

    del train_df,valid_df
    gc.collect()
    
    ip_count = df.groupby(['ip','device','os','app'])['channel'].count().reset_index()
    ip_count.columns = ['ip','device','os','app', 'clicks_by_ip_os_device_app']
    gc.collect()
    df = pd.merge(df, ip_count, on=['ip','device','os','app'], how='left', sort=False)
    #valid_df = pd.merge(valid_df, ip_count, on=['ip','device','os','app'], how='left', sort=False)
    del ip_count
    gc.collect()
    print('done')
    
   
    ip_count = df.groupby(['ip','device'])['channel'].count().reset_index()
    ip_count.columns = ['ip','device','clicks_by_ip_device']
    gc.collect()
    df = pd.merge(df, ip_count, on=['ip','device'], how='left', sort=False)
    #valid_df = pd.merge(valid_df, ip_count, on=['device','os','app'], how='left', sort=False)
    #df['clicks_by_os_device_app'] = df['clicks_by_os_device_app'].astype('uint16')
    #valid_df['clicks_by_os_device_app'] = valid_df['clicks_by_os_device_app'].astype('uint16')
    del ip_count
    gc.collect()
    print('done')
    
    
    ip_count = df.groupby(['ip','device','os'])['app'].agg(['count']).reset_index()
    ip_count.columns = ['ip','device','os','count_app_by_iposdevice']
    df = pd.merge(df, ip_count, on=['ip','device','os'], how='left', sort=False)
    #valid_df = pd.merge(valid_df, ip_count, on=['ip','device','os'], how='left', sort=False)
    del ip_count
    gc.collect()

    # Count the number of clicks by ip
    ip_count = df.groupby(['ip'])['channel'].count().reset_index()
    ip_count.columns = ['ip', 'clicks_by_ip']
    gc.collect()
    df = pd.merge(df, ip_count, on='ip', how='left', sort=False)
    #valid_df = pd.merge(valid_df, ip_count, on='ip', how='left', sort=False)
    #df['clicks_by_ip'] = df['clicks_by_ip'].astype('uint16')
    #valid_df['clicks_by_ip'] = valid_df['clicks_by_ip'].astype('uint16')
    del ip_count
    gc.collect()
    print('done')

    ip_count = df.groupby(['ip','app'])['channel'].count().reset_index()
    ip_count.columns = ['ip','app', 'clicks_by_ip_app']
    gc.collect()
    df = pd.merge(df, ip_count, on=['ip','app'], how='left', sort=False)
    #df['clicks_by_ip_app'] = df['clicks_by_ip_app'].astype('uint16')
    #valid_df = pd.merge(valid_df, ip_count, on=['ip','app'], how='left', sort=False)
    #valid_df['clicks_by_ip_app'] = valid_df['clicks_by_ip_app'].astype('uint16')
    del ip_count
    gc.collect()
    print('done')

    ip_count = df.groupby(['ip','os'])['channel'].count().reset_index()
    ip_count.columns = ['ip','os', 'clicks_by_ip_os']
    df = pd.merge(df, ip_count, on=['ip','os'], how='left', sort=False)
    #df['clicks_by_ip_os'] = df['clicks_by_ip_os'].astype('uint16')
    #valid_df = pd.merge(valid_df, ip_count, on=['ip','os'], how='left', sort=False)
    #valid_df['clicks_by_ip_os'] = valid_df['clicks_by_ip_os'].astype('uint16')
    gc.collect()
    del ip_count
    

    train_df = df[:nrow_train]
    valid_df = df[nrow_train:]

    del df
    print('done')
    #merge.drop('ip', axis=1, inplace=True)
    gc.collect()
    return train_df,valid_df


CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 10.5 µs


In [12]:
%%time
gc.collect()
train,valid=get_feature_count(train,valid)
print(valid.shape,train.shape)

done
done
done
done
done
(53016937, 15) (56983063, 15)
CPU times: user 7min 39s, sys: 1min 17s, total: 8min 56s
Wall time: 8min 56s


In [18]:
gc.collect()
train.head()

,ip,app,device,os,channel,click_time,is_attributed,dow,hour,clicks_by_ip_os_device_app,clicks_by_ip_device,count_app_by_iposdevice,clicks_by_ip,clicks_by_ip_app,clicks_by_ip_os
0,87242,24,1,40,178,2017-11-08 01:47:40,0,2,1,1,963,12,970,30,12
1,49565,12,1,9,259,2017-11-08 01:47:40,0,2,1,14,408,37,408,71,37
2,39785,9,1,13,244,2017-11-08 01:47:40,0,2,1,152,5464,1388,5504,638,1404
3,5348,3,1,18,280,2017-11-08 01:47:40,0,2,1,5365,606118,28403,700322,120694,31202
4,56516,9,1,17,466,2017-11-08 01:47:40,0,2,1,14,3272,217,3320,319,217


In [20]:

def get_feature_user(df,valid_df):
        
    ip_count = df.groupby(['ip','device','os'])['is_attributed'].agg(['mean']).reset_index()
    ip_count.columns = ['ip','device','os', 'mean_attr_by_iposdevice']
    df = pd.merge(df, ip_count, on=['ip','device','os'], how='left', sort=False)
    valid_df = pd.merge(valid_df, ip_count, on=['ip','device','os'], how='left', sort=False)
    del ip_count
    gc.collect()
    
    agg_cols = ['device', 'app', 'channel','ip','os'];
    for c in agg_cols:
        gc.collect()
        gp = df.groupby(c)['is_attributed']
        mean = gp.mean()
        gc.collect()
        df[c + '_attr_avg'] = df[c].map(mean)
        valid_df[c + '_attr_avg'] = valid_df[c].map(mean)
        
    
    del mean,gp,agg_cols
    gc.collect()
    return df,valid_df
    

In [21]:
%%time
train,valid=get_feature_user(train,valid)
print(valid.shape,train.shape)

(53016937, 21) (56983063, 21)
CPU times: user 2min 41s, sys: 26.8 s, total: 3min 8s
Wall time: 3min 8s


In [ ]:
'''
%%time
ip_count = merge.groupby(['ip','dow','hour'])['channel'].count().reset_index()
ip_count.columns = ['ip','dow','hour','clicks_by_ip_dow_hour']
merge = pd.merge(merge, ip_count, on=['ip','dow','hour'], how='left', sort=False)
merge['clicks_by_ip_dow_hour'] = merge['clicks_by_ip_dow_hour'].astype('uint16')
del ip_count
gc.collect()

'''

In [ ]:
'''
%%time

# Count the number of os by ip
ip_count = merge.groupby(['channel'])['os'].count().reset_index()
ip_count.columns = ['channel', 'clicks_by_channel']
merge = pd.merge(merge, ip_count, on='channel', how='left', sort=False)
merge['clicks_by_channel'] = merge['clicks_by_channel'].astype('uint16')
#merge.drop('channel', axis=1, inplace=True)
del ip_count
print('channel done')

ip_count = merge.groupby(['os'])['channel'].count().reset_index()
ip_count.columns = ['os', 'clicks_by_os']
merge = pd.merge(merge, ip_count, on='os', how='left', sort=False)
merge['clicks_by_os'] = merge['clicks_by_os'].astype('uint16')
#merge.drop('os', axis=1, inplace=True)
del ip_count
print('os done')
gc.collect()

ip_count = merge.groupby(['app'])['channel'].count().reset_index()
ip_count.columns = ['app', 'clicks_by_app']
merge = pd.merge(merge, ip_count, on='app', how='left', sort=False)
merge['clicks_by_app'] = merge['clicks_by_app'].astype('uint16')
#merge.drop('app', axis=1, inplace=True)
del ip_count

print('app done')
#del ip_count,merge
#gc.collect()
'''

In [ ]:
'''

train = merge[:nrow_train]
test = merge[nrow_train:]

del merge

gc.collect()
'''

In [ ]:
#test.head()

In [23]:
gc.collect()

0

In [24]:
train.head()

,ip,app,device,os,channel,click_time,is_attributed,dow,hour,clicks_by_ip_os_device_app,...,count_app_by_iposdevice,clicks_by_ip,clicks_by_ip_app,clicks_by_ip_os,mean_attr_by_iposdevice,device_attr_avg,app_attr_avg,channel_attr_avg,ip_attr_avg,os_attr_avg
0,87242,24,1,40,178,2017-11-08 01:47:40,0,2,1,1,...,12,970,30,12,0.000000,0.001754,0.000470,0.000109,0.000000,0.003006
1,49565,12,1,9,259,2017-11-08 01:47:40,0,2,1,14,...,37,408,71,37,0.000000,0.001754,0.000105,0.000329,0.000000,0.000734
2,39785,9,1,13,244,2017-11-08 01:47:40,0,2,1,152,...,1388,5504,638,1404,0.001433,0.001754,0.001120,0.001905,0.001027,0.001482
3,5348,3,1,18,280,2017-11-08 01:47:40,0,2,1,5365,...,28403,700322,120694,31202,0.000726,0.001754,0.000244,0.000216,0.001697,0.001368
4,56516,9,1,17,466,2017-11-08 01:47:40,0,2,1,14,...,217,3320,319,217,0.000000,0.001754,0.001120,0.001138,0.001247,0.001123


In [25]:
%%time
train["click_time"] = pd.to_datetime(train["click_time"], format="%Y-%m-%d %H:%M:%S")
valid["click_time"] = pd.to_datetime(valid["click_time"], format="%Y-%m-%d %H:%M:%S")
gc.collect()
train.head()

CPU times: user 27.4 s, sys: 1.02 s, total: 28.4 s
Wall time: 28.1 s


In [26]:
%%time
train["ordinal_date"] = train["click_time"].apply(lambda x: time.mktime(x.timetuple()))
gc.collect()
valid["ordinal_date"] = valid["click_time"].apply(lambda x: time.mktime(x.timetuple()))
gc.collect()

CPU times: user 19min 23s, sys: 3min 14s, total: 22min 37s
Wall time: 22min 28s


In [27]:
train["train_set"] = 1
valid["train_set"] = 0
gc.collect()

14

In [28]:
%%time
train = train.sort_values(by="ordinal_date").reset_index(drop=True)
valid = valid.sort_values(by="ordinal_date").reset_index(drop=True)
gc.collect()

CPU times: user 28.3 s, sys: 1min 12s, total: 1min 41s
Wall time: 1min 43s


In [31]:
train.head()

,ip,app,device,os,channel,click_time,is_attributed,dow,hour,clicks_by_ip_os_device_app,...,clicks_by_ip_app,clicks_by_ip_os,mean_attr_by_iposdevice,device_attr_avg,app_attr_avg,channel_attr_avg,ip_attr_avg,os_attr_avg,ordinal_date,train_set
0,87242,24,1,40,178,2017-11-08 01:47:40,0,2,1,1,...,30,12,0.0,0.001754,0.000470,0.000109,0.000000,0.003006,1.510106e+09,1
1,188573,18,1,20,439,2017-11-08 01:47:40,0,2,1,1,...,61,11,0.0,0.001754,0.000507,0.000427,0.000000,0.001296,1.510106e+09,1
2,81514,2,2,13,401,2017-11-08 01:47:40,0,2,1,21,...,1175,1947,0.0,0.000290,0.000251,0.000229,0.000126,0.001482,1.510106e+09,1
3,35707,21,1,19,128,2017-11-08 01:47:40,0,2,1,45,...,169,1266,0.0,0.001754,0.000142,0.000231,0.000277,0.001767,1.510106e+09,1
4,74324,2,1,17,237,2017-11-08 01:47:40,0,2,1,174,...,1387,892,0.0,0.001754,0.000251,0.000222,0.001590,0.001123,1.510106e+09,1


In [32]:
%%time
gc.collect
full_df = train.append(valid).reset_index(drop=True)

CPU times: user 16.4 s, sys: 1min 13s, total: 1min 29s
Wall time: 1min 49s


In [33]:
full_df.shape

(110000000, 23)

In [34]:
del train,valid
gc.collect()

234

In [35]:
%%time
full_df["user_prev_date"] = full_df.groupby(['ip','device','os'])["ordinal_date"].shift(1)
full_df["user_date_diff"] = full_df["ordinal_date"] - full_df["user_prev_date"]

CPU times: user 30.8 s, sys: 4.63 s, total: 35.4 s
Wall time: 36.9 s


In [37]:
gc.collect()
full_df.head()

,ip,app,device,os,channel,click_time,is_attributed,dow,hour,clicks_by_ip_os_device_app,...,mean_attr_by_iposdevice,device_attr_avg,app_attr_avg,channel_attr_avg,ip_attr_avg,os_attr_avg,ordinal_date,train_set,user_prev_date,user_date_diff
0,87242,24,1,40,178,2017-11-08 01:47:40,0,2,1,1,...,0.0,0.001754,0.000470,0.000109,0.000000,0.003006,1.510106e+09,1,NaN,NaN
1,188573,18,1,20,439,2017-11-08 01:47:40,0,2,1,1,...,0.0,0.001754,0.000507,0.000427,0.000000,0.001296,1.510106e+09,1,NaN,NaN
2,81514,2,2,13,401,2017-11-08 01:47:40,0,2,1,21,...,0.0,0.000290,0.000251,0.000229,0.000126,0.001482,1.510106e+09,1,NaN,NaN
3,35707,21,1,19,128,2017-11-08 01:47:40,0,2,1,45,...,0.0,0.001754,0.000142,0.000231,0.000277,0.001767,1.510106e+09,1,NaN,NaN
4,74324,2,1,17,237,2017-11-08 01:47:40,0,2,1,174,...,0.0,0.001754,0.000251,0.000222,0.001590,0.001123,1.510106e+09,1,NaN,NaN


In [38]:
'''
%%time
gdf = full_df.groupby(['ip','device','os'])["ordinal_date"].agg([ "mean", "std"]).reset_index()
gdf.columns = ["ip","device","os","user_mean_date","user_std_date"]
full_df = pd.merge(full_df, gdf, on=['ip','device','os'])
del gdf
gc.collect()


'''

CPU times: user 3min 28s, sys: 1min 23s, total: 4min 52s
Wall time: 5min 34s


In [39]:
print(full_df.shape)
full_df.head()

(110000000, 29)


,ip,app,device,os,channel,click_time,is_attributed,dow,hour,clicks_by_ip_os_device_app,...,ip_attr_avg,os_attr_avg,ordinal_date,train_set,user_prev_date,user_date_diff,user_min_date,user_mean_date,user_max_date,user_std_date
0,87242,24,1,40,178,2017-11-08 01:47:40,0,2,1,1,...,0.0,0.003006,1.510106e+09,1,NaN,NaN,1.510106e+09,1.510112e+09,1.510113e+09,2873.611628
1,87242,1,1,40,125,2017-11-08 01:47:40,0,2,1,2,...,0.0,0.003006,1.510106e+09,1,1.510106e+09,0.0,1.510106e+09,1.510112e+09,1.510113e+09,2873.611628
2,87242,21,1,40,128,2017-11-08 03:49:51,0,2,3,2,...,0.0,0.003006,1.510113e+09,1,1.510106e+09,7331.0,1.510106e+09,1.510112e+09,1.510113e+09,2873.611628
3,87242,14,1,40,123,2017-11-08 03:50:04,0,2,3,1,...,0.0,0.003006,1.510113e+09,1,1.510113e+09,13.0,1.510106e+09,1.510112e+09,1.510113e+09,2873.611628
4,87242,8,1,40,145,2017-11-08 03:50:49,0,2,3,3,...,0.0,0.003006,1.510113e+09,1,1.510113e+09,45.0,1.510106e+09,1.510112e+09,1.510113e+09,2873.611628


In [40]:
%%time
gdf = full_df[full_df['is_attributed']==1]
gdf = gdf.groupby(['ip','device','os'])["ordinal_date"].agg(["mean","std"]).reset_index()
gdf.columns = ["ip","device","os","user_mean_date_click", "user_std_date_click"]
full_df = pd.merge(full_df, gdf, on=['ip','device','os'], how="left")
del gdf
gc.collect()

CPU times: user 40.3 s, sys: 1min 11s, total: 1min 52s
Wall time: 2min 46s


In [41]:
full_df.head()

,ip,app,device,os,channel,click_time,is_attributed,dow,hour,clicks_by_ip_os_device_app,...,ordinal_date,train_set,user_prev_date,user_date_diff,user_min_date,user_mean_date,user_max_date,user_std_date,user_mean_date_click,user_std_date_click
0,87242,24,1,40,178,2017-11-08 01:47:40,0,2,1,1,...,1.510106e+09,1,NaN,NaN,1.510106e+09,1.510112e+09,1.510113e+09,2873.611628,NaN,NaN
1,87242,1,1,40,125,2017-11-08 01:47:40,0,2,1,2,...,1.510106e+09,1,1.510106e+09,0.0,1.510106e+09,1.510112e+09,1.510113e+09,2873.611628,NaN,NaN
2,87242,21,1,40,128,2017-11-08 03:49:51,0,2,3,2,...,1.510113e+09,1,1.510106e+09,7331.0,1.510106e+09,1.510112e+09,1.510113e+09,2873.611628,NaN,NaN
3,87242,14,1,40,123,2017-11-08 03:50:04,0,2,3,1,...,1.510113e+09,1,1.510113e+09,13.0,1.510106e+09,1.510112e+09,1.510113e+09,2873.611628,NaN,NaN
4,87242,8,1,40,145,2017-11-08 03:50:49,0,2,3,3,...,1.510113e+09,1,1.510113e+09,45.0,1.510106e+09,1.510112e+09,1.510113e+09,2873.611628,NaN,NaN


In [42]:
full_df = full_df.drop(["ordinal_date", "click_time", "user_prev_date"], axis=1)

In [43]:
gc.collect()
train = full_df[full_df["train_set"] == 1]
valid = full_df[full_df["train_set"] == 0]
train.shape,valid.shape

((56983063, 28), (53016937, 28))

In [44]:
del full_df
gc.collect()

0

In [ ]:
'''
def get_deltatime(df):
    GROUP_BY_NEXT_CLICKS = [
        {'groupby': ['ip','device','os']},
       ]

    # Calculate the time to next click for each group
    for spec in GROUP_BY_NEXT_CLICKS:

        # Name of new feature
        new_feature = '{}_nextClick'.format('_'.join(spec['groupby']))    

        # Unique list of features to select
        all_features = spec['groupby'] + ['click_time']

        # Run calculation
        print(f">> Grouping by {spec['groupby']}, and saving time to next click in: {new_feature}")
        df[new_feature] = df.groupby(spec['groupby']).click_time.transform(lambda x: x.diff().shift(-1)).dt.seconds
        
    df.drop(['click_time'],axis=1,inplace=True)
    return df

'''

In [47]:
train.drop(['ip','train_set'],axis=1,inplace=True)
valid.drop(['ip','train_set'],axis=1,inplace=True)
print(train.shape,valid.shape)
gc.collect()

(56983063, 26) (53016937, 26)


0

In [ ]:
#test.reset_index(inplace=True,drop=True)
#valid.drop(['is_attributed'],axis=1,inplace=True)
#gc.collect()

In [67]:
gc.collect()

0

In [69]:
%%time
train.to_feather('train.ftr')
valid.to_feather('valid.ftr')

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 8.11 µs


In [ ]:
%%time
gc.collect()
train=pd.read_feather('train.ftr')
valid=pd.read_feather('valid.ftr')

In [92]:
#train.drop(['user_min_date','user_mean_date','user_max_date'],axis=1,inplace=True)
train.drop(['dow', 'hour','clicks_by_ip_app','clicks_by_ip_app', 'clicks_by_ip_os'],axis=1,inplace=True)



In [93]:
#valid.drop(['user_min_date','user_mean_date','user_max_date'],axis=1,inplace=True)
valid.drop(['dow', 'hour','clicks_by_ip_app','clicks_by_ip_app', 'clicks_by_ip_os'],axis=1,inplace=True)

gc.collect()

3390

In [94]:
    
dtrain = lgbm.Dataset(train.drop(['is_attributed'],axis=1), label=train.is_attributed)
dvalid = lgbm.Dataset(valid.drop(['is_attributed'],axis=1), label=valid.is_attributed)

In [ ]:
#del train,valid

In [95]:
gc.collect()

0

In [ ]:

params = {
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': 'auc',
    'learning_rate': 0.01,
    'num_leaves':30,  
    'max_depth': 6,
    'nthread': 4,
    'reg_alpha': 0.1, # L1 regularization term on weights
    'reg_lambda': 0.1,
    #'scale_pos_weight': 99.7
    'verbose':1
    }


evals_results = {}

print("Training the model...")

lgb_model = lgbm.train(params, dtrain,valid_sets=[dtrain, dvalid], 
                 valid_names=['train','valid'], 
                 evals_result=evals_results, 
                 num_boost_round=20)

Training the model...
[1]	train's auc: 0.936836	valid's auc: 0.835805
[2]	train's auc: 0.938461	valid's auc: 0.837497
[3]	train's auc: 0.938462	valid's auc: 0.837497
[4]	train's auc: 0.938462	valid's auc: 0.837497
[5]	train's auc: 0.938463	valid's auc: 0.837498
[6]	train's auc: 0.938463	valid's auc: 0.837497


In [ ]:
import matplotlib.pyplot as plt
ax = lgbm.plot_importance(lgb_model, max_num_features=30)
plt.show()

In [ ]:
print('Feature names:', lgb_model.feature_name())
print('Feature importances:', list(lgb_model.feature_importance()))


In [ ]:
submit = pd.read_csv(path_test, dtype='int', usecols=['click_id'])
submit['is_attributed'] = lgb_model.predict(test[predictors], num_iteration=lgb_model.best_iteration)
submit.to_csv('submission.csv', index=False)


In [ ]:

del train
del val
gc.collect()
